In [11]:
import pandas as pd
from viame_annotation import Viame
import os

In [8]:
def viame_to_standard(csv_path, source):
    # Load the CSV file
    df = pd.read_csv(csv_path, skiprows=lambda x: x in [1])
    viame = Viame()

    # Initialize a list to hold the data for each row
    rows_list = []

    # Iterate over the rows of the DataFrame and process each annotation
    for index, row in df.iterrows():
        # Build the Filename
        filename = viame.get_id(row, source)
        # Extract Family, Genus, Species
        family, genus, species = viame.get_taxonomy(row)
        
        # Extract bounding box coordinates
        xmin, ymin, xmax, ymax = viame.get_bbox(row)
        
        # Prepare the new row as a Series
        new_row = pd.Series({
            'Filename': filename,
            'Family': family,
            'Genus': genus,
            'Species': species,
            'ymin': ymin,
            'xmin': xmin,
            'xmax': xmax,
            'ymax': ymax,
            'Augmentation': "none",
            'Source': source
        })

        # Append the new Series to the list
        rows_list.append(new_row)

    # Concatenate all the Series into a new DataFrame
    converted_df = pd.concat(rows_list, axis=1).transpose()

    # Write the converted DataFrame to a new CSV file
    return converted_df

In [ ]:
def standard_to_viame(standard_df, original_csv_path, video_folder):
  # TODO: THIS IS CHATGPT. CHECK BACK
    # Read the original CSV to get the column names
    original_df = pd.read_csv(original_csv_path, nrows=0)

    # Create a new DataFrame with the same columns as the original
    viame_df = pd.DataFrame(columns=original_df.columns)

    # Iterate over the rows of the standard DataFrame and convert each back to the original format
    for index, row in standard_df.iterrows():
        # Rebuild the '10-11+: Repeated Species' field
        repeated_species = f"{row['Genus']} {row['Species']}"
        
        # Rebuild the '4-7: Img-bbox(TL_x,TL_y,BR_x,BR_y)' field
        tl_x = row['xmin']
        tl_y = row['ymin']
        br_x = row['xmax']
        br_y = row['ymax']
        bbox = f"{tl_x},{tl_y},{br_x},{br_y}"
        
        # Rebuild the '3: Unique Frame Identifier' field
        frame_id = int(row['Filename'].split('_frame')[1].split('.')[0]) // 30
        
        # Prepare the new row
        new_row = {
            '1: Detection or Track-id': "",  # Fill in or calculate as needed
            '2: Video or Image Identifier': video_folder,  # Assuming video_folder is equivalent to this field
            '3: Unique Frame Identifier': frame_id,
            '4-7: Img-bbox(TL_x,TL_y,BR_x,BR_y)': bbox,
            '8: Detection or Length Confidence': "",  # Fill in or calculate as needed
            '9: Target Length (0 or -1 if invalid)': 0,  # Assuming default value
            '10-11+: Repeated Species,Confidence Pairs or Attributes': repeated_species
            # Add additional columns as needed
        }

        # Append the new row to the DataFrame
        viame_df = viame_df.append(new_row, ignore_index=True)
    
    # Return the converted DataFrame
    return viame_df

# USAGE

In [12]:
# Example usage of the function
annotation_folder = '/vol/biomedic3/bglocker/ugproj2324/fv220/datasets/youtube_bruvs/annotations_viame'
output_folder = '/vol/biomedic3/bglocker/ugproj2324/fv220/datasets/youtube_bruvs/annotations_sharktrack'

for annotation in os.listdir(annotation_folder):
  if annotation.endswith(".csv"):
    annotation_name = annotation.split('.')[0]
    print(f"Converting {annotation}")
    # Convert the CSV file
    standard_df = viame_to_standard(os.path.join(annotation_folder, annotation), annotation_name)
    # Write the converted DataFrame to a new CSV file
    standard_df.to_csv(os.path.join(output_folder, annotation), index=False)
    print(f"Converted {annotation}")

Converting gfp_niue1.csv
Converted gfp_niue1.csv
Converting gfp_maldives1.csv
Converted gfp_maldives1.csv
Converting gfp_tobago1.csv
Converted gfp_tobago1.csv
Converting gfp_tiger1.csv
Converted gfp_tiger1.csv
Converting gfp_caledonia1.csv
Converted gfp_caledonia1.csv
Converting gfp_rand10.csv
Converted gfp_rand10.csv
Converting gfp_tonga1.csv
Converted gfp_tonga1.csv
Converting gfp_png1.csv
Converted gfp_png1.csv
Converting gfp_polynesia1.csv
Converted gfp_polynesia1.csv
Converting gfp_rand3.csv
Converted gfp_rand3.csv
Converting gfp_kiribati1.csv
Converted gfp_kiribati1.csv
Converting gfp_rand4.csv
Converted gfp_rand4.csv
Converting gfp_belize1.csv
Converted gfp_belize1.csv
Converting gfp_hawaii1.csv
Converted gfp_hawaii1.csv
Converting gfp_bahamas1.csv
Converted gfp_bahamas1.csv
Converting gfp_montserrat1.csv
Converted gfp_montserrat1.csv
Converting gfp_rand5.csv
Converted gfp_rand5.csv
Converting gfp_rand2.csv
Converted gfp_rand2.csv
Converting gfp_rand11.csv
Converted gfp_rand11.c

In [10]:
converted.head()

,Filename,Family,Genus,Species,ymin,xmin,xmax,ymax,Augmentation,Source
0,gfp_bahamas1_frame120.jpg,,shark,,98,11,1278,718,none,gfp_bahamas1
1,gfp_bahamas1_frame150.jpg,,shark,,81,1,1269,717,none,gfp_bahamas1
2,gfp_bahamas1_frame420.jpg,,shark,,72,18,211,147,none,gfp_bahamas1
3,gfp_bahamas1_frame540.jpg,,shark,,0,0,977,513,none,gfp_bahamas1
4,gfp_bahamas1_frame570.jpg,,shark,,0,12,1268,321,none,gfp_bahamas1
